In [1]:
# # to work with devstral model:
# !pip install mistral-common --upgrade
# !pip install git+https://github.com/mistralai/mistral-common.git

# from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
# from huggingface_hub import hf_hub_download

# model_id = "mistralai/Devstral-Small-2505"
# tekken_file = hf_hub_download(repo_id=model_id, filename="tekken.json")

# tokenizer = MistralTokenizer.from_file(tekken_file)

# imports

In [2]:
import json
import logging
import pandas as pd
from transformers import AutoTokenizer
from datasets import load_dataset, concatenate_datasets

/opt/conda/envs/grok/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Create a logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Remove existing handlers
logger.handlers.clear()

# Add a simple handler
handler = logging.StreamHandler()
formatter = logging.Formatter('[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

# utils

In [4]:
# def process_dataset(n, token_counts, sd, df, tokenizer, prompt_template, prompt_keys_mapping):
#     """
#     Process the dataset and convert it to JSON Lines format.
    
#     :param n: number of dataset samples
#     :param token_counts: list of target prompt token counts (e.g., [128, 256])
#     :param s: std-like tolerance window (e.g., 10 means [x-10, x+10])
#     :param df: pandas dataframe object
#     :param tokenizer: tokenizer object with .encode method
#     :param prompt_template: string template with placeholders like {input}, {context}
#     :param prompt_keys_mapping: list of dicts mapping prompt keys to dataset column names
#     :return: dict of {target_token_len: DataFrame} for each len in `token_counts`

#     """

#     key_map = {m["prompt_key"]: m["dataset_key"] for m in prompt_keys_mapping}
    
#     df = df.sample(n=min(n, len(df))).reset_index(drop=True)
#     logger.info(f"dataset shape after sampling: {df.shape}")
    
    
#     df["prompt"] = df.apply(
#         lambda row:  prompt_template.format(**{k: row[v] for k, v in key_map.items()}),
#         axis=1
#     )
#     df["prompt_tc"] = df["prompt"].apply(lambda x: len(tokenizer.encode(x)))
    
#     df = df[["prompt", "prompt_tc"]]
#     # Create a dictionary of filtered DataFrames for each length in token_counts
#     results = {}
#     for tc in token_counts:
#         lower, upper = tc - sd, tc + sd
#         filtered = df[
#             (df["prompt_tc"] >= lower) & 
#             (df["prompt_tc"] <= upper)
#         ].copy().reset_index(drop=True)
#         logger.info(f"token_count {tc}: {len(filtered)} samples")
#         results[str(tc)] = filtered

#     return results

# dataset 1: general

## smoltalk

In [5]:
dataset_hf_id = "HuggingFaceTB/smoltalk"
subset_names = ["all"] # ["explore-instruct-rewriting", "apigen-80k", "smol-rewrite", "smol-summarize", "longalign", "self-oss-instruct"]
split = "train[:100]" # "test"

dss = [] 
for name in subset_names:
    ds = load_dataset(
        path=dataset_hf_id, 
        name=name,
        split=split
    )
    # ds = ds.add_column("source", [name] * len(ds)) # NOTE: use is using subsets
    dss.append(ds)
    print(f"len({name}): {len(ds)}")

smoltalk = concatenate_datasets(dss)
original_columns = smoltalk.features.keys()

print("-"*25)
print(f"len(smoltalk): {len(smoltalk)}")
print(f"columns: {original_columns}")
print("-"*25)

len(all): 100
-------------------------
len(smoltalk): 100
columns: dict_keys(['messages', 'source'])
-------------------------


In [42]:
smoltalk[10]["messages"]

[{'content': 'Provide a concise, objective summary of the input text in up to three sentences, focusing on key actions and intentions without using second or third person pronouns.',
  'role': 'system'},
 {'content': "Emily,\n\nIt's fantastic to hear about the successful treatment of the infant's heat rash! I'm glad my advice was helpful, and I'm always happy to collaborate with a fellow pediatrician who shares my passion for addressing skin conditions in children.\n\nI'm definitely still interested in working together on the heat rash prevention project. It's an important topic, and I believe our combined expertise can make a real difference.\n\nThank you for the warm invitation to meet up during my visit to your city. I'll be arriving on the 15th and staying until the 20th. I would love to take you up on your offer to show me around and sample some local cuisine. Perhaps we could meet on the 17th or 18th?\n\nPlease let me know which date works best for you, and I'll make the necessar

#### gemma

In [7]:
hf_model_id = "google/gemma-3-1b-it"
gemma_tokenizer = AutoTokenizer.from_pretrained(hf_model_id)

print(gemma_tokenizer.chat_template)

# formatted_prompt = gemma_tokenizer.apply_chat_template(smoltalk[1000]["messages"], tokenize=False, add_generation_prompt=True)
# print(formatted_prompt)

{{ bos_token }}
{%- if messages[0]['role'] == 'system' -%}
    {%- if messages[0]['content'] is string -%}
        {%- set first_user_prefix = messages[0]['content'] + '

' -%}
    {%- else -%}
        {%- set first_user_prefix = messages[0]['content'][0]['text'] + '

' -%}
    {%- endif -%}
    {%- set loop_messages = messages[1:] -%}
{%- else -%}
    {%- set first_user_prefix = "" -%}
    {%- set loop_messages = messages -%}
{%- endif -%}
{%- for message in loop_messages -%}
    {%- if (message['role'] == 'user') != (loop.index0 % 2 == 0) -%}
        {{ raise_exception("Conversation roles must alternate user/assistant/user/assistant/...") }}
    {%- endif -%}
    {%- if (message['role'] == 'assistant') -%}
        {%- set role = "model" -%}
    {%- else -%}
        {%- set role = message['role'] -%}
    {%- endif -%}
    {{ '<start_of_turn>' + role + '
' + (first_user_prefix if loop.first else "") }}
    {%- if message['content'] is string -%}
        {{ message['content'] | trim }}


In [ ]:
def construct_prompt(message, tokenizer):
    convo = message["messages"]
    try:
        prompt = tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=True)
        return {
            "prompt_tc": len(tokenizer.encode(prompt)),
            "prompt": prompt,
            "output": convo[-1]["content"] if convo and convo[-1]["role"] == "assistant" else ""
        }
    except Exception as e:
        logger.error(f"ErrorMsg: {e}")
    return {
        "prompt_tc": None,
        "prompt": None,
        "output": None
    }

# 33 mins
smoltalk_processed = smoltalk.map(lambda message: construct_prompt(message, gemma_tokenizer), remove_columns=original_columns)
smoltalk_processed

Map: 100%|██████████| 100/100 [00:00<00:00, 443.19 examples/s]


Dataset({
    features: ['prompt_tc', 'prompt', 'output'],
    num_rows: 100
})

In [22]:
smoltalk_processed.to_json("gemma-smoltalk.jsonl", lines=True)

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 231.01ba/s]


516994

In [23]:
# smoltalk[140000]

In [10]:
# print(smoltalk_processed[140000]["prompt"])

## sharegpt

In [66]:
json_dataset_path = "raw/ShareGPT_V3_unfiltered_cleaned_split.json"

with open(json_dataset_path, "r") as f:
    sharegpt = json.load(f)

sharegpt = pd.DataFrame(sharegpt)
    
print(f"len(sharegpt): {len(sharegpt)}")

len(sharegpt): 94145


In [67]:
sharegpt[:1]

,id,conversations
0,QWJhYvA_0,"[{'from': 'human', 'value': 'Summarize the mai..."


In [80]:
def update(convo):
    role_map = {
        "human": "user",
        "gpt": "assistant",
    }
    # print(convo)
    convo = [
        {
            "role": role_map.get(m["from"], "user"), 
            "content": m["value"]
        } 
        for m in convo
    ]
    return convo

# for idx, row in sharegpt.iterrows():
#     sharegpt.iloc[idx]["converations"] = update(row["conversations"])
    
sharegpt["conversations"] = sharegpt["conversations"].apply(lambda row: update(row["conversations"]))

TypeError: list indices must be integers or slices, not str

In [76]:
print(sharegpt.iloc[0]["conversations"])

[{'from': 'human', 'value': "Summarize the main ideas of Jeff Walker's Product Launch Formula into bullet points as it pertains to a growth marketing agency implementing these strategies and tactics for their clients..."}, {'from': 'gpt', 'value': "Here are the main ideas of Jeff Walker's Product Launch Formula that can be applied by a growth marketing agency for their clients:\n\n1. Identify the target audience and their needs: Understand the ideal customer for the product or service, and create a messaging that resonates with them.\n2. Pre-launch: Build anticipation and excitement for the launch by creating buzz, gathering testimonials and case studies, and using social media to create awareness.\n3. Launch: Use a well-crafted launch sequence to maximize sales and conversions. This can include offering bonuses, creating scarcity, and using a deadline to create urgency.\n4. Post-launch: Follow up with customers, gather feedback, and continue to provide value to keep them engaged and l

### gemma

In [24]:
hf_model_id = "google/gemma-3-1b-it"
gemma_tokenizer = AutoTokenizer.from_pretrained(hf_model_id)

# print(gemma_tokenizer.chat_template)

In [92]:
def convert_roles(convo, human_key, assistant_key):
    role_map = {
        human_key: "user",
        assistant_key: "assistant",
    }
    convo = [
        {
            "role": role_map.get(m["from"], "user"), 
            "content": m["value"]
        } 
        for m in convo
    ]
    
    try:
        if len(convo)>=1 and (not convo[0]["role"] == "user"):
            return convo[1:]
    except Exception as e:
        logger.error(f"Errorsg: {e}")
        # logger.info(f"convo: {convo}")
    return convo


def process_sharegpt_row(row, tokenizer):
    convo = convert_roles(convo=row["conversations"] , human_key="human", assistant_key="gpt")
    
    try:
        output = ""
        if len(convo) >= 1:
            prompt = tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=True)
            if convo[-1]["role"] == "assistant":
                output = convo[-1]["content"] 
        
            return pd.Series({
                "prompt_tc": len(tokenizer.encode(prompt)),
                "prompt": prompt,
                "output": output,
             })
    except Exception as e:
        logger.error(f"ErrorMsg: {e}")
        # logger.info(f"convo: {convo}")
        # for m in convo:
        #     print(m["role"])
        # print(f"-"*25)
    return pd.Series({
        "prompt_tc": None,
        "prompt": None,
        "output": None
    })
    
# sharegpt[:10].apply(lambda row: process_sharegpt_row(row, gemma_tokenizer), axis=1)

# 33 mins
sharegpt_processed = sharegpt.apply(lambda row: process_sharegpt_row(row, gemma_tokenizer), axis=1)
sharegpt_processed.head()

[2025-06-09 12:32:15,466] p22403 {1863983978.py:39} ERROR - ErrorMsg: Conversation roles must alternate user/assistant/user/assistant/...
[2025-06-09 12:32:17,523] p22403 {1863983978.py:39} ERROR - ErrorMsg: Conversation roles must alternate user/assistant/user/assistant/...
[2025-06-09 12:32:17,524] p22403 {1863983978.py:39} ERROR - ErrorMsg: Conversation roles must alternate user/assistant/user/assistant/...
[2025-06-09 12:32:18,766] p22403 {1863983978.py:39} ERROR - ErrorMsg: Conversation roles must alternate user/assistant/user/assistant/...
[2025-06-09 12:32:19,255] p22403 {1863983978.py:39} ERROR - ErrorMsg: Conversation roles must alternate user/assistant/user/assistant/...
[2025-06-09 12:32:19,738] p22403 {1863983978.py:39} ERROR - ErrorMsg: Conversation roles must alternate user/assistant/user/assistant/...
[2025-06-09 12:32:19,971] p22403 {1863983978.py:39} ERROR - ErrorMsg: Conversation roles must alternate user/assistant/user/assistant/...
[2025-06-09 12:32:22,638] p22403 {

,prompt_tc,prompt,output
0,1932.0,<bos><start_of_turn>user\nSummarize the main i...,Tony Robbins describes six core human needs th...
1,104.0,<bos><start_of_turn>user\nHow to tell if a cus...,1. Homogeneity: The segment should consist of ...
2,523.0,"<bos><start_of_turn>user\nIn Java, I want to r...",You can use the `String.format()` method in Ja...
3,2209.0,<bos><start_of_turn>user\nMetaphorical languag...,Lo and behold! By the grace of divine interven...
4,1807.0,<bos><start_of_turn>user\nMore verbose and det...,"Behold, fellow seeker of knowledge, for I shal..."


In [90]:
sum(sharegpt_processed["prompt"].isna())

1

In [ ]:
# def format_conversation_prompt(conversation: list[dict], prompt_template) -> str:
#     # Split context and question
#     context_parts = []
#     final_question = ""
#     for i, turn in enumerate(conversation):
#         role = turn["from"]
#         content = turn["value"]

#         if role == "human":
#             if i == len(conversation) - 2:
#                 final_question = content  # last user turn
#             else:
#                 context_parts.append(f"\nQuestion: {content}")
#         elif role == "gpt":
#             context_parts.append(f"\nAnswer: {content}")

#     context = "\n".join(context_parts)

#     # Final prompt
#     prompt = prompt_template.format(
#         context=context,
#         input=final_question
#     )

#     return prompt



# with open("/home/ubuntu/grok-llms/inference/arena/prompt-templates/sharegpt.txt", 'r') as file:
#     sharegpt_pt = file.read()

# # prompt = format_conversation_prompt(
# #     conversation=sharegpt[0]["conversations"], prompt_template=sharegpt_pt
# # )
# # print(prompt)


# sharegpt_df = pd.DataFrame()
# sharegpt_df["source"] = "sharegpt"
# sharegpt_df["prompt"] = [
#     format_conversation_prompt(conversation=conv["conversations"], prompt_template=sharegpt_pt)
#     for conv in sharegpt
# ]
# sharegpt_df.shape

## longbench

In [ ]:
datasets = [
    # "hotpotqa",
    # "triviaqa",
    "triviaqa_e"
]

for dataset in datasets:
    data = load_dataset("THUDM/LongBench", dataset, split="test")
    
data

In [ ]:
print(
    " ---------- input ----------\n",
    data[0]["input"],
    "---------- context ----------\n",
    data[0]["context"],
    "---------- answers ----------\n",
    data[0]["answers"],
)


# dataset 2: reasoning